**Week4: Part 3 - Segmenting and Clustering Neighborhoods in Toronto**

**Import Libraries**

In [5]:

from bs4 import BeautifulSoup
import requests
import urllib
import urllib.request

**Convert the URL to HTML and parse into list**

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')
#print(soup.prettify())

In [7]:
table = soup.find('table', class_ = 'wikitable sortable')
#print(match.prettify())

header = table.tbody.tr.th.text
print(header)

Postcode


In [8]:
for table in soup.find_all('table', class_ = 'wikitable sortable'):
    header = table.tbody.tr.text
    print(header)
header = header.rstrip('\n')
header = header.lstrip('\n')
header = header.replace('\n', ',')
header_list = header.split(',')
header_list


Postcode
Borough
Neighbourhood



['Postcode', 'Borough', 'Neighbourhood']

In [9]:
entry_list = []
count = 0
for table in soup.find_all('table', class_ = 'wikitable sortable'):
    entry = table.tbody.text
    entry = entry.rstrip('\n')
    entry = entry.rstrip(' ')
    entry = entry.lstrip('\n')
    entry = entry.lstrip(' ')
    entry = entry.replace('\n', ',')
    
    entry_list.append(entry)
    entry_list = entry.split(',')
    


In [10]:
for element in entry_list:
    try:
        entry_list.remove('')
    except ValueError:
        pass

del entry_list[0:3]
entry_list[0:10]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A']

**Import needed libraries so that data can be converted to pandas dataframe**

In [11]:
import pandas as pd
import numpy as np
df_toronto = pd.DataFrame(data = np.array(entry_list).reshape(289,3), columns = header_list)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Date Cleaning and Segmenting**

In [12]:
df_toronto.replace('Not assigned', np.nan, inplace=True)
df_toronto = df_toronto.dropna(subset=['Borough'])
print(df_toronto.shape)
df_toronto.head(10)

(212, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [13]:
df_toronto.fillna(0, inplace=True)
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,0
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [14]:
mask = df_toronto['Neighbourhood'] == 0
df_toronto.loc[mask, 'Neighbourhood'] = df_toronto['Borough']
df_toronto['Neighbourhood'] = df_toronto['Neighbourhood'].mask(mask, df_toronto['Borough'])
print(df_toronto.shape)
df_toronto.head(10)

(212, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [15]:
df_toronto_grouped = df_toronto.groupby(['Postcode', 'Borough']).agg({'Neighbourhood':lambda x: ', '.join(x)}).reset_index()
df_toronto_grouped.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
df_toronto_grouped

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [17]:
df_toronto_grouped.shape

(103, 3)

**Week 4 Part 2: Getting the Latitude and Longitude for each Neighborhood**

**Read csv file and store as pandas dataframe**


In [18]:
url2 = 'http://cocl.us/Geospatial_data'
df_ll = pd.read_csv(url2)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Rename columns headers and then merge dataframes**

In [19]:
df_ll.rename({'Postal Code':'Postcode'}, axis=1, inplace=True)
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_toronto_ll = pd.merge(df_toronto_grouped, df_ll, on='Postcode')
df_toronto_ll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Week 4 Part 3: Clustering Neighborhoods of Toronto**

**Import Needed libraries**

In [21]:
from geopy.geocoders import Nominatim
import folium
import json
import requests
from pandas.io.json import json_normalize

**We will be looking at the neighborhoods that are in a borough that contain the word 'Toronto' Not Including Central Toronto**


In [22]:
toronto_east = df_toronto_ll[df_toronto_ll['Borough'] == 'East Toronto'].reset_index(drop=True)
toronto_west = df_toronto_ll[df_toronto_ll['Borough'] == 'West Toronto'].reset_index(drop=True)
toronto_DT = df_toronto_ll[df_toronto_ll['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
frames = [toronto_east, toronto_west, toronto_DT]
toronto_hoods = pd.concat(frames).set_index('Postcode').reset_index()
toronto_hoods
toronto_hoods

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
5,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
6,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
7,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
8,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
9,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325


In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Visualization**

In [24]:
# Now a map can be made of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)


for lat, lng, label in zip(toronto_hoods['Latitude'], toronto_hoods['Longitude'], toronto_hoods['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  
    
map_toronto